In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import os
import gc
from tqdm import tqdm
import random

import warnings
warnings.filterwarnings('ignore')

In [2]:
import keras
import tensorflow as tf
from tensorflow.keras import optimizers, callbacks
from tensorflow.keras.layers import Dense, Concatenate, Activation, Add, BatchNormalization, Dropout, Input
from tensorflow.keras.models import Model, Sequential, load_model

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)

def mish(x):
    return x*tf.math.tanh(tf.math.softplus(x))

def decay(epochs):
    init = 1e-3
    drop = 10
    ratio = 0.9
    return max(5e-5, (init * (ratio ** (epochs//drop))))

es = callbacks.EarlyStopping(patience=10, restore_best_weights=True)
lrs = callbacks.LearningRateScheduler(decay, verbose=0)


Using TensorFlow backend.


In [3]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

data = load_breast_cancer()
data, target = data['data'], data['target']

tr_X, val_X, tr_y, val_y = train_test_split(data, target, test_size=0.2, random_state=SEED)

In [4]:
scaler = StandardScaler()
scaler.fit(tr_X)

tr_X = scaler.transform(tr_X)
val_X = scaler.transform(val_X)

In [5]:
def DenseBlock(n, activation=None, bn=True):
    def f(x):
        x = Dense(n, kernel_initializer='he_normal')(x)
        if bn:
            x = BatchNormalization()(x)
        x = Activation(activation)(x)
        return x
    return f


def build_NN(input_shape):
    inputs = Input(shape=input_shape)
    
    x = DenseBlock(32, activation=mish)(inputs)
    x = DenseBlock(16, activation=mish)(x)
    
    outputs = Dense(1, activation='sigmoid')(x)
    
    return Model(inputs, outputs)

In [6]:
nn = build_NN(tr_X.shape[1:])
nn.compile(loss='binary_crossentropy', optimizer=optimizers.Adam())

In [7]:
nn.fit(tr_X, tr_y,
      epochs=150,
      validation_split=0.15,
      callbacks=[es, lrs])

Train on 386 samples, validate on 69 samples
Epoch 1/150
386/386 [==============================] - 1s 2ms/sample - loss: 0.4917 - val_loss: 0.3358
Epoch 2/150
386/386 [==============================] - 0s 220us/sample - loss: 0.3929 - val_loss: 0.2777
Epoch 3/150
386/386 [==============================] - 0s 225us/sample - loss: 0.3263 - val_loss: 0.2425
Epoch 4/150
386/386 [==============================] - 0s 189us/sample - loss: 0.2735 - val_loss: 0.2230
Epoch 5/150
386/386 [==============================] - 0s 207us/sample - loss: 0.2552 - val_loss: 0.2078
Epoch 6/150
386/386 [==============================] - 0s 205us/sample - loss: 0.2386 - val_loss: 0.1961
Epoch 7/150
386/386 [==============================] - 0s 171us/sample - loss: 0.2150 - val_loss: 0.1859
Epoch 8/150
386/386 [==============================] - 0s 187us/sample - loss: 0.2005 - val_loss: 0.1776
Epoch 9/150
386/386 [==============================] - 0s 183us/sample - loss: 0.1907 - val_loss: 0.1711
Epoch 10/150

In [8]:
roc_auc_score(val_y, nn.predict(val_X))

0.9944317065181788